<a href="https://colab.research.google.com/github/evinracher/3010090-ontological-engineering/blob/main/Taller_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller 1

###**Objetivos de aprendizaje**
- Tokenizacion
- Embeddings
- Transformes
- Huggingface
- Modelo Bert

## **Tokenización**

Uno de los componentes claves para trabajar con transformes es la tokenización, esta tarea se enfoca en particionar las frases en palabras con el fin de generar un **input** para diferentes tareas de PLN

Veamos con la libreria transformers como podemos tokenizar...

In [ ]:
from transformers import AutoTokenizer

# Usaremos BERT como modelo
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

texto = "I love learning about Transformers!"

inputs = tokenizer(texto)

# El modelo asigna un numero a cada token conocido
print(f"Input IDs: {inputs['input_ids']}")

# Decodificación para ver qué entiende el modelo
print(f"Texto decodificado: {tokenizer.decode(inputs['input_ids'])}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Input IDs: [101, 1045, 2293, 4083, 2055, 19081, 999, 102]
Texto decodificado: [CLS] i love learning about transformers! [SEP]


¿Y si usamos otro modelo? *¿por qué cambia los ids de las palabras?*

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("openai-community/gpt2")

texto = "I love learning about Transformers!"

inputs = tokenizer(texto)

# El modelo asigna un numero a cada token conocido
print(f"Input IDs: {inputs['input_ids']}")

# Decodificación para ver qué entiende el modelo
print(f"Texto decodificado: {tokenizer.decode(inputs['input_ids'])}")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Input IDs: [40, 1842, 4673, 546, 39185, 0]
Texto decodificado: I love learning about Transformers!


###Ejercicio
Tokeniza las siguientes palabras usando el modelo bert

In [ ]:
Texto1 = "I'm a good developer"
Texto2 = "El sol brillaba intensamente sobre las antiguas ruinas mayas"
Texto3 = "La música de la orquesta llenaba la sala con melodías mágicas"

# SOLUCION

inputs1 = tokenizer(Texto2)

# El modelo asigna un numero a cada token conocido
print(f"Input IDs: {inputs1['input_ids']}")

# Decodificación para ver qué entiende el modelo
print(f"Texto decodificado: {tokenizer.decode(inputs1['input_ids'])}")

Input IDs: [9527, 1540, 9547, 15498, 17509, 3263, 68, 523, 4679, 39990, 1885, 328, 84, 292, 16866, 292, 743, 292]
Texto decodificado: El sol brillaba intensamente sobre las antiguas ruinas mayas


## **Embeddings**

De Palabras a Vectores

En lugar de representar una palabra con un simple ID (como Manzana = 5), un embedding la representa como una lista de números (un vector). Cada número en esa lista representa una "dimensión" o característica semántica, aunque no siempre sepamos exactamente qué significa cada una.

- Palabra: "Rey"
- Vector (Simplificado): [0.9, 0.1, 0.8] (Donde las dimensiones podrían ser [Masculinidad, Juventud, Realeza]).

La Geometría del Significado

La potencia real de los embeddings es que palabras con significados similares terminan en posiciones cercanas en este espacio matemático.
- "Perro" y "Cachorro" estarán muy cerca.
- "Perro" y "Computadora" estarán muy lejos.

Esto permite realizar "aritmética de palabras".

El ejemplo más famoso es:$$\vec{Rey} - \vec{Hombre} + \vec{Mujer} \approx \vec{Reina}$$

Anteriormente vimos un vector que contenia la representación de la frase en un vector de numeros... ¿Es esto un embedding?

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
texto = "I love Transformers"
inputs = tokenizer(texto)
print(f"Embedding: {inputs['input_ids']}")
print(f"Texto decodificado: {tokenizer.decode(inputs['input_ids'])}")

Embedding: [101, 1045, 2293, 19081, 102]
Texto decodificado: [CLS] i love transformers [SEP]


Los embeddings no se escriben a mano; se aprenden mediante el entrenamiento de redes neuronales (como Word2Vec, GloVe o los mismos Transformers). El modelo analiza millones de textos y aprende que si "naranja" y "manzana" suelen aparecer cerca de la palabra "comer" o "jugo", entonces deben tener características similares.

### Caracteristicas de un Embedding

1. Densidad (Density)

A diferencia de representaciones antiguas como el One-Hot Encoding (donde tendrías un vector de miles de ceros y un solo uno), los embeddings son vectores densos.
- Significado: Casi todos los valores en el vector son números decimales (distintos de cero).
- Ventaja: Permiten almacenar mucha más información en un espacio mucho más pequeño (por ejemplo, un vector de 512 o 768 dimensiones para representar millones de palabras).
2. Dimensionalidad Fija

Cada modelo define un tamaño específico para sus vectores de embedding ($d_{model}$).
- En modelos base de BERT, la dimensión suele ser 768.
- En modelos más grandes como GPT-3, puede llegar a 12,288.
- Importante: Independientemente de si la palabra es "a" o "electroencefalografista", ambas tendrán un vector del mismo tamaño.
3. Relación Semántica (Cercanía Vectorial)

Esta es la característica más "mágica". Los embeddings posicionan palabras con significados o usos similares en áreas cercanas del espacio vectorial.
- Cercanía: "Coche" y "Automóvil" tendrán una distancia euclidiana mínima.
- Direccionalidad: Las relaciones entre palabras se conservan como desplazamientos geométricos (ej. la distancia de "París" a "Francia" es similar a la de "Madrid" a "España").

4. Capacidad de Generalización

Al no tratar a las palabras como símbolos aislados, el modelo puede entender palabras que no vio frecuentemente en el entrenamiento si están cerca de otras que sí conoce. Si el modelo sabe qué es "delicioso", y "exquisito" tiene un embedding cercano, el modelo sabrá cómo reaccionar ante "exquisito" aunque sea la primera vez que lo lee.

5. Contextualidad (Solo en Transformers)

Esta es la característica que separa a los embeddings antiguos (Word2Vec) de los modernos (BERT, Gemini, GPT):
- Embeddings Estáticos: La palabra "hoja" (de papel) y "hoja" (de árbol) tendrían el mismo vector.
- Embeddings Dinámicos: El Transformer ajusta el vector de la palabra basándose en las palabras que la rodean. El vector de "hoja" cambia si la frase habla de una impresora o de un bosque.

###Gensim

Este programa proporciona un ejemplo práctico de cómo utilizar la biblioteca Gensim para crear incrustaciones de palabras. Estos modelos tienen la capacidad de encontrar palabras similares basándose en sus representaciones vectoriales resalta el poder de las incrustaciones de palabras para capturar relaciones semánticas.

In [2]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 24.5 MB/s eta 0:00:00


Cargaremos el modelo pre entrenado...

In [3]:
import gensim.downloader as api

# Lista de modelos pre entrenados
#print(list(api.info()['models'].keys()))

# Cargamos el modelo pre entrenado con este data set
wv = api.load('glove-wiki-gigaword-50')

[==================================================] 100.0% 66.0/66.0MB downloaded


#### Tareas con embeddings

Vector para una palabra

In [ ]:
vector = wv['language']
print("Vector de representation de la palabra 'language':")
print(vector)
print("Tamaño:", len(vector))

# El vector de representación de la palabra "language" es recobrado desde el modelo pre entrenado.

Vector de representation de la palabra 'language':
[-5.7990e-01 -1.1010e-01 -1.1557e+00 -2.9906e-03 -2.0613e-01  4.5289e-01
 -1.6671e-01 -1.0382e+00 -9.9241e-01  3.9884e-01  5.9230e-01  2.2990e-01
  1.5213e+00 -1.7764e-01 -2.9726e-01 -3.9235e-01 -7.8471e-01  1.5594e-01
  6.9077e-01  5.9537e-01 -4.4340e-01  5.3514e-01  3.2853e-01  1.2437e+00
  1.2972e+00 -1.3878e+00 -1.0925e+00 -4.0925e-01 -5.6971e-01 -3.4656e-01
  3.7163e+00 -1.0489e+00 -4.6708e-01 -4.4739e-01  6.2300e-03  1.9649e-02
 -4.0161e-01 -6.2913e-01 -8.2506e-01  4.5591e-01  8.2626e-01  5.7091e-01
  2.1199e-01  4.6865e-01 -6.0027e-01  2.9920e-01  6.7944e-01  1.4238e+00
 -3.2152e-02 -1.2603e-01]
Tamaño: 50


Palabras similares

In [ ]:
similar_words = wv.most_similar('language')
print("Most similar words to 'language':")
print(similar_words)

# Se usa el método most_similar para identificar las palabras mas similares

Most similar words to 'language':
[('languages', 0.8814865946769714), ('word', 0.8100197315216064), ('spoken', 0.8074647784233093), ('vocabulary', 0.7903235554695129), ('translation', 0.7879166007041931), ('arabic', 0.7812612056732178), ('refers', 0.7787709832191467), ('dialect', 0.7587653994560242), ('literature', 0.7525033950805664), ('english', 0.7479498982429504)]


La palabra que no pertenece a un conjunto

In [ ]:
print(wv.doesnt_match(['fire', 'water', 'land', 'sea', 'air', 'car']))

car


Palabras parecidas a un conjunto de palabras

In [ ]:
print(wv.most_similar(positive=['car', 'minivan'], topn=5))

[('truck', 0.9100273251533508), ('suv', 0.904007613658905), ('jeep', 0.8619828820228577), ('vehicle', 0.8431736826896667), ('cars', 0.8421834111213684)]


Aritmética de palabras

In [ ]:
print(wv.most_similar(positive=['woman','king'], negative=['man'], topn = 3))

[('queen', 0.8523604273796082), ('throne', 0.7664334177970886), ('prince', 0.7592144012451172)]


###Ejercicio

Realiza las siguientes tareas con las palabras
1. Las 3 palabras mas similares a "work"
2. Del conjunto que palabra no pertenece `["monday", "tuesday", "friday", "blue"]`
3. Resultado de la aritmetica de palabras $\vec{Paris} - \vec{Berlin} + \vec{Germany} \approx \vec{?}$


In [6]:
print("Punto 1: ", wv.most_similar('work', topn=3))
print("Punto 2: ", wv.doesnt_match(["monday", "tuesday", "friday", "blue"]))

result3 = wv.most_similar(positive=['paris','germany'], negative=['berlin'], topn = 1)[0][0]
print("Punto 3: ", result3)

Punto 1:  [('working', 0.9163011908531189), ('done', 0.8910592794418335), ('well', 0.8780409693717957)]
Punto 2:  blue
Punto 3:  france


## Transformers

Es un modelo de red neuronal diseñado para procesar secuencias de datos (como texto, audio o series temporales) de manera extremadamente eficiente. Fue introducido por Google en 2017 en el famoso artículo "Attention is All You Need" y es la tecnología que hace posible a herramientas como ChatGPT, Gemini y traductores modernos.

Que deberia tener presente al estudiar acerca de los transformers:

- Estudia el paper original "Attention Is All You Need" (2017).

- Positional Encoding: Cómo el modelo sabe el orden de las palabras sin usar RNNs.

- Encoder vs. Decoder: Entender qué bloques se usan para tareas de comprensión (BERT) y cuáles para generación (GPT).

- Layer Normalization y Residual Connections: Técnicas que permiten entrenar modelos muy profundos.

Antes de 2017, y antes de los Transformers, usábamos redes recurrentes (RNN) que leían palabra por palabra (de izquierda a derecha). Los Transformers cambiaron esto al "mirar" toda la secuencia al mismo tiempo.

### Pilares del Transformer

- El Mecanismo de Atención (Self-Attention):
Es la capacidad del modelo para decidir qué palabras de una oración son importantes para entender otra palabra específica.
- Procesamiento en Paralelo:
A diferencia de sus predecesores, el Transformer no necesita leer en orden. Procesa todos los tokens simultáneamente. Esto permite que el entrenamiento sea muchísimo más rápido usando hardware moderno (GPUs).

- Codificación Posicional (Positional Encoding): Como el modelo lee todo a la vez, no sabe intrínsecamente en qué orden están las palabras. Para solucionar esto, el Transformer añade una señal matemática a cada palabra que indica su posición exacta en la frase.

### La Arquitectura: Encoder y Decoder
Originalmente, el Transformer se divide en dos grandes bloques:

- Encoder (Codificador): Lee el texto de entrada y lo convierte en una representación abstracta (embeddings contextuales). Modelos como BERT usan solo el Encoder.

- Decoder (Decodificador): Toma esa representación y genera una secuencia de salida palabra por palabra. Modelos como GPT usan principalmente el Decoder.

##[Hugging face](https://huggingface.co/)
Hugging Face es una empresa y plataforma de inteligencia artificial que se ha convertido en el ecosistema más popular para trabajar con modelos de lenguaje (NLP) y otros modelos de deep learning.

Su misión es democratizar la inteligencia artificial, es decir, hacer que cualquier persona pueda usar, entrenar y compartir modelos de IA fácilmente.

### Libreria transformers

La librería `transformers` es el corazón técnico de Hugging Face.
Es un paquete de Python que te permite usar modelos preentrenados de última generación con solo unas pocas líneas de código.

Algunas de sus funcionalidades:

- Acceso a miles de modelos (BERT, XLNet, GPT, RoBERTa, etc.)
- Carga automática de modelos y herramientas auxiliares como tokenizadores
- Ejecución en CPU o GPU
- Fine-tuning (entrenar con tus propios datos)
- Exportar y compartir modelos fácilmente

¿Que otras tareas diferentes a PLN se pueden ejecutar usando la libreria?

### Plataforma Hugging face

Hugging face es mucho más que documentación:
es una plataforma colaborativa al estilo GitHub, pero para modelos de IA. En ella podemos encontrar:
- Repositorios de modelos preentrenados
- Datasets
- Aplicaciones listas para usar
- Entrenar y evaluar modelos
- Compartir tus propios modelos
- Documentación y ejemplos	Tutoriales, notebooks y guías prácticas para principiantes y expertos

¿Cómo puedes compartir tu propio modelo entrenado en Hugging Face Hub?

## Modelo [BERT](https://huggingface.co/docs/transformers/model_doc/bert)

Bidirectional Encoder Representations from Transformers, es uno de los modelos de inteligencia artificial más influyentes en el campo del Procesamiento de Lenguaje Natural (NLP). Fue desarrollado por Google en 2018 y cambió las reglas del juego porque permitió que las máquinas entendieran el lenguaje de una forma mucho más humana y contextual.

1. ¿Qué lo hace especial? La Bidireccionalidad

Antes de BERT, los modelos leían el texto de izquierda a derecha o de derecha a izquierda, pero no ambas al mismo tiempo.

-  El problema: Si lees "El banco está...", no sabes si es un asiento o una entidad financiera hasta que terminas la frase.

- La solución de BERT: Analiza la frase completa en ambas direcciones simultáneamente. Esto le permite entender el significado de una palabra basándose en todo su entorno (lo que viene antes y lo que viene después).

2. Basado solo en el "Encoder"

A diferencia del Transformer original (que tiene un Encoder para leer y un Decoder para escribir), BERT utiliza únicamente el bloque Encoder.

- Su objetivo principal no es generar texto (como GPT), sino entenderlo y representarlo en vectores numéricos (embeddings) extremadamente precisos. Por eso es el rey en tareas de clasificación, búsqueda y análisis de sentimientos.

3. ¿Cómo aprendió? Sus dos tareas de entrenamiento

Google entrenó a BERT con todo Wikipedia y miles de libros usando dos métodos ingeniosos:

- MLM (Masked Language Model): El modelo oculta palabras al azar en una oración (les pone una "máscara") y trata de adivinar cuáles son. Esto lo obliga a entender la gramática y la semántica. Ejemplo: "El gato se subió al [MASK] para dormir". BERT predice "techo" o "sofá".

- NSP (Next Sentence Prediction): El modelo recibe dos oraciones y debe decidir si la segunda es la continuación lógica de la primera. Esto le ayuda a entender la relación entre párrafos y documentos largos.

4. Transfer Learning y Fine-tuning

Una de las mayores ventajas de BERT es que no tienes que entrenarlo desde cero.

- Pre-training: Google ya hizo el trabajo pesado de enseñarle el idioma.

- Fine-tuning: Tú tomas ese modelo ya "educado" y le das un pequeño entrenamiento adicional con tus propios datos (por ejemplo, correos electrónicos de tu empresa) para que aprenda una tarea específica, como detectar spam o clasificar tickets de soporte.

Aplicación: Analisis de sentimientos

In [7]:
from transformers import pipeline
clf = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment")
print(clf("This movie is awful"))
print(clf("I like the movie"))

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

[{'label': '1 star', 'score': 0.9448572397232056}]
[{'label': '4 stars', 'score': 0.5495378971099854}]


Aplicación: NER

¿A qué hace referencia los parametros que se envian en la función pipeline  en la libreria transformers?

In [ ]:
ner = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple")
text = "El presidente Pedro Sánchez viajó a Madrid el lunes."
print(ner(text))

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

¿Que otras aplicaciones podemos crear usando el modelo BERT?

###Ejercicio
Realiza una implementacion usando el modelo Bert de las siguientes tareas:
- question-answering(preguntas al texto)
- fill mask(completar el texto)

In [ ]:
question = pipeline("question-answering", model="dslim/bert-base-NER", aggregation_strategy="simple")
text = "El presidente Pedro Sánchez viajó a Madrid el lunes."
print(ner(text))

---

FACULTAD DE MINAS<br>
Sede Medellín

SINTELWEB<br>
Grupo de Investigación<br>
Sistemas Inteligentes